In [11]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from pytorchvideo.data.encoded_video import EncodedVideo
from transformers import BertTokenizer
from torchvision.transforms import Compose, Lambda, Resize, Normalize, ColorJitter

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)

### Initial Transform

In [2]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 8
sampling_rate = 8
frames_per_second = 30

# Note that this transform is specific to the slow_R50 model.
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(8),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size=(crop_size, crop_size))
        ]
    ),
)

### Sampling frames per clip divided over the video

In [3]:
video_root = '../train'
index = 0
data = pd.read_csv('../train.csv')


In [4]:
data.iloc[index]['video_id']+".mp4"

'KabnUV5luJ8.mp4'

In [5]:
video_path = os.path.join(video_root, data.iloc[index]['video_id']+".mp4")
label = data.iloc[index]['label']
text = data.iloc[index]['text']

In [6]:
print(f'video_path: {video_path}')

video_path: ../train/KabnUV5luJ8.mp4


In [7]:
print(f'label: {label}')

label: 1


In [8]:
print(f'text: {text}')

text: please name of the software deep fake try putting great thunberg on hitler true it wud be hard to tell dude you are too good cuz she looks like melania trump


In [9]:
# Load video using PyTorchVideo
video = EncodedVideo.from_path(video_path)

In [10]:
print(video)

In [11]:
# Get video duration and calculate the step size for frame sampling
duration = video.duration
step = duration / num_frames
print(f'duration: {duration}')
print(f'step: {step}')

duration: 561152/11025
step: 70144/11025


In [12]:
# Sample frames at regular intervals
video_data = []
for i in range(num_frames):
    start_sec = i * step
    end_sec = start_sec + step
    clip = video.get_clip(start_sec=start_sec, end_sec=end_sec)
    video_data.append(transform(clip))

KeyboardInterrupt: 

In [ ]:
print(len(video_data))

32


In [ ]:
print(video_data[0]['video'].shape)

torch.Size([3, 1, 256, 256])


In [ ]:
# Stack the sampled frames
# Extract video tensors from each dictionary
video_tensors = [item['video'] for item in video_data]

# Stack video tensors along the frames dimension
stacked_video = torch.stack(video_tensors)

In [ ]:
stacked_video = stacked_video.squeeze(2).permute(1, 0, 2, 3)

In [ ]:
stacked_video.shape

torch.Size([3, 32, 256, 256])

In [ ]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True)

Using cache found in /home/davendra/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
input = stacked_video.unsqueeze(0)
input.shape

torch.Size([1, 3, 32, 256, 256])

In [ ]:
pred = model(input)
pred.shape

torch.Size([1, 400])

In [ ]:
feature_extractor = torch.nn.Sequential(*list(model.blocks.children())[:-1])

In [ ]:
pred2 = feature_extractor(input)
pred2.shape

torch.Size([1, 2048, 32, 8, 8])

In [ ]:
pred3 = pred2.permute(0, 2, 3, 4, 1)
pred3.shape

torch.Size([1, 32, 8, 8, 2048])

### Sampling over the entire video

In [ ]:
video_root = '../train'
index = 0
data = pd.read_csv('../train.csv')

In [ ]:
video_path = os.path.join(video_root, data.iloc[index]['video_id']+".mp4")
label = data.iloc[index]['label']
text = data.iloc[index]['text']

In [ ]:
print(f'video_path: {video_path}')

video_path: ../train/KabnUV5luJ8.mp4


In [ ]:
print(f'label: {label}')

label: 1


In [ ]:
print(f'text: {text}')

text: please name of the software deep fake try putting great thunberg on hitler true it wud be hard to tell dude you are too good cuz she looks like melania trump


In [ ]:
clip_duration = (num_frames * sampling_rate)/frames_per_second
start_sec = 0
end_sec = start_sec + clip_duration

# Initialize an EncodedVideo helper class and load the video
video = EncodedVideo.from_path(video_path)

print(video)

In [ ]:
# Load the desired clip
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

print(video_data['video'].shape)

# Apply a transform to normalize the video input
video_data = transform(video_data)

print(video_data['video'].shape)

torch.Size([3, 64, 720, 1280])
torch.Size([3, 8, 256, 256])


### Dataset Object

In [45]:
class DeepFakeDataset(Dataset):
    def __init__(self, video_path_file, text_csv_file, text_transforms=None, video_transforms=None, num_frames=8, sampling_rate=8, frames_per_second=30):
        # self.root = video_root
        self.video_annotation = pd.read_csv(video_path_file)
        self.text_df = pd.read_csv(text_csv_file)
        self.text_transforms = text_transforms
        self.video_transforms = video_transforms
        self.num_frames = num_frames
        self.sampling_rate = sampling_rate
        self.frames_per_second = frames_per_second

    def __len__(self):
        return len(self.video_annotation)

    def __getitem__(self, index):

        video_path = self.video_annotation.iloc[index]['video_path']
        label = self.video_annotation.iloc[index]['label']
        text = self.text_df.iloc[index]['text']

        try:
            # Load video using PyTorchVideo
            video = EncodedVideo.from_path(video_path)

            # Get video duration and calculate the step size for frame sampling
            duration = video.duration
            step = duration / self.num_frames

            # Sample frames at regular intervals
            video_data = []
            for i in range(self.num_frames):
                start_sec = i * step
                end_sec = start_sec + step
                clip = video.get_clip(start_sec=start_sec, end_sec=end_sec)
                print(f'clip shape: {clip['video'].shape}')
                video_data.append(self.video_transforms(clip['video']))

            # Stack the sampled frames
            video_data = torch.stack(video_data)
            # print(video_data)
            print(f'Video_data shape: {video_data.shape}')
            
            # Stack the sampled frames
            # Extract video tensors from each dictionary
            # video_tensors = [item['video'] for item in video_data]

            # Stack video tensors along the frames dimension
            # video_frames = torch.stack(video_tensors).squeeze(2).permute(1, 0, 2, 3)

            # clip_duration = min((num_frames * sampling_rate) / frames_per_second, video.duration)
            # start_sec = 0
            # end_sec = start_sec + clip_duration

            # video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

            # video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

            # Apply a transform to normalize the video input
            # if self.video_transforms:
            #     video_frames = self.video_transforms(video_data['video'])

        except Exception as e:
            print(f'Error Processing video {video_path}: {e}')
            # print(f'Clip Duration: {clip_duration}')
            # print(f'Video Duration: {video.duration}')

        # Apply text transforms
        # if self.text_transforms:
        #     text_data = self.text_transforms(text)
        # else:
        
        text_data = text

        return {
            'video': video_data,
            'text': text_data,
            'label': torch.tensor(label, dtype=torch.long)
        }

### Example using Dataset

In [46]:
# Create instances of text and video transforms
text_transforms = Compose([
    BertTokenizer.from_pretrained('bert-base-uncased'),
    # Add more text transformations as needed
])

# video_transforms = Compose([
#     ApplyTransformToKey(
#         key="video",
#         transform=Compose([
#             UniformTemporalSubsample(num_video_samples=1),
#             RandomResizedCrop(size=224, scale=(0.8, 1.0), ratio=(0.75, 1.33)),
#             RandomHorizontalFlip(p=0.5),
#             ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
#             # Add more video transformations as needed
#             Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#         ]),
#     ),
# ])

side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 30
sampling_rate = 8
frames_per_second = 30

# Note that this transform is specific to the slow_R50 model.
# video_transforms =  ApplyTransformToKey(
#     key="video",
#     transform=Compose(
#         [
#             UniformTemporalSubsample(num_frames),
#             Lambda(lambda x: x/255.0),
#             NormalizeVideo(mean, std),
#             ShortSideScale(
#                 size=side_size
#             ),
#             CenterCropVideo(crop_size=(crop_size, crop_size))
#         ]
#     ),
# )

video_transforms = Compose(
        [
            UniformTemporalSubsample(30),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size=(crop_size, crop_size))
        ]
    )

# Create an instance of the dataset
video_path_file = '../annotations/video_train_path.csv'
text_csv_file = '../annotations/text_train.csv'

dataset = DeepFakeDataset(
                            video_path_file=video_path_file,
                            text_csv_file=text_csv_file,
                            text_transforms=text_transforms,
                            video_transforms=video_transforms,
                            num_frames=num_frames,
                            sampling_rate=sampling_rate,
                            frames_per_second=frames_per_second
                        )

In [47]:
# Create a dataloader
batch_size = 4
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [48]:
cnt = 0
video_train=list()
for data in dataloader:
    video_train.append(data['video'])

    cnt += 1

    if cnt == 3:
        break

clip shape: torch.Size([3, 137, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 137, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 137, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 137, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 136, 720, 1280])
clip shape: torch.Size([3, 137, 720, 1280])
clip shape: torch.Size([3, 136, 

mmco: unref short failure


clip shape: torch.Size([3, 160, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 160, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 160, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 160, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 160, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 160, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 160, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 160, 720, 1280])
clip shape: torch.Size([3, 161, 720, 1280])
clip shape: torch.Size([3, 160, 720, 1280])
clip shape: torch.Size([3, 160, 720, 1280])
Video_data shape: torch.Size([30, 3, 30, 256, 256])


In [49]:
video_train[0].shape

torch.Size([4, 30, 3, 30, 256, 256])